### Importing Libraires and CSV

In [72]:
import pandas as pd
import random
import os.path
AM_data = pd.read_csv('Peru_2019_AudioMoth_Data_Full.csv')

/var/folders/69/lf6vfqbd7wxb691hjhqbw0980000gn/T/ipykernel_696/2280191482.py:4: DtypeWarning: Columns (1,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  AM_data = pd.read_csv('Peru_2019_AudioMoth_Data_Full.csv')


### Dataset with only minute long clips

In [73]:
AM_data['Duration'].unique
AM_data_minute = AM_data[AM_data['Duration']>=60.0]
AM_data = AM_data_minute


### Dataset with only Audiomoths with more than 24 clips

In [74]:
b = AM_data['AudioMothCode'].value_counts()
for audiomoth, counts in b.items():
    if (counts < 24):
        AM_data = AM_data.drop(AM_data[AM_data['AudioMothCode'] == audiomoth].index)
AM_data = AM_data.reset_index(drop=True)

### Add column for hour

In [75]:
AM_data = AM_data.dropna(subset=['StartDateTime'])
hours = []
i=0
while i < AM_data['StartDateTime'].size:
    time = AM_data.iloc[i].get('StartDateTime')
    hour = time.split(' ')[1].split(':')[0]
    hour = int(hour)
    hours.append(hour)
    i+=1

AM_data.insert(0,"hour",hours)

### dataframe with one valid Audio Moth per hour

In [76]:
list_AM_hour = []
list_AM = AM_data['AudioMothCode'].unique()
currhour = 0
for currAM in list_AM:
    for currhour in range(24):
        dh = AM_data[(AM_data['hour'] == currhour) & (AM_data['AudioMothCode'] == currAM)].index
        for i in dh:
            list_AM_hour.append(i)
        random_idx = random.randint(0,len(list_AM_hour)-1)
        list_AM_hour.pop(random_idx)
        AM_data.drop(index=list_AM_hour,inplace=True)
        AM_data = AM_data.reset_index(drop=True)
        list_AM_hour = []   
AM_data.drop(columns='hour',inplace=True)

### Convert to CSV

In [77]:
AM_data.to_csv('Stratified_Peru_2019_AudioMoth_Data_Full.csv',index=False)